# Example 3: Spatial Interpolation
## Setup

Install / load required dependencies.

In [ ]:
import os
import datetime
import sys
import requests
from urllib.request import urlretrieve
import urllib.request, json 

import torch
import torchvision.transforms as transforms
import torchvision
import torch.nn as nn
import torch.nn.functional as F

from decimal import Decimal, getcontext
from torch.utils.data import TensorDataset, DataLoader

import numpy as np
import pandas as pd
import scipy

import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import normalize
from sklearn import metrics

Check GPU device availability.

In [ ]:
torch.cuda.get_device_name(0)

Helper functions, as found in `src/utils.py`.

In [ ]:
#Normalize data values. Default is [0,1] range; if min_val = -1, range is [-1,1]
def normal(tensor,min_val=0):
  t_min = torch.min(tensor)
  t_max = torch.max(tensor)
  if t_min == 0 and t_max == 0:
    return torch.tensor(tensor)
  if min_val == -1:
    tensor_norm = 2 * ((tensor - t_min) / (t_max - t_min)) - 1
  if min_val== 0:
    tensor_norm = ((tensor - t_min) / (t_max - t_min))
  return torch.tensor(tensor_norm)

#Light-weight Local Moran's I for tensor data, requiring a sparse weight matrix input. 
#This can be used when there is no need to re-compute the weight matrix at each step
def lw_tensor_local_moran(y,w_sparse,na_to_zero=True,norm=True,norm_min_val=0):
  y = y.reshape(-1)
  n = len(y)
  n_1 = n - 1
  z = y - y.mean()
  sy = y.std()
  z /= sy
  den = (z * z).sum()
  zl = torch.tensor(w_sparse * z)
  mi = n_1 * z * zl / den
  if na_to_zero==True:
    mi[torch.isnan(mi)] = 0
  if norm==True:
    mi = normal(mi,min_val=norm_min_val)
  return torch.tensor(mi)

#Batch version of lw_tensor_local_moran
#Computes the (normalized) local Moran's I for an input batch
def batch_lw_tensor_local_moran(y_batch,w_sparse,na_to_zero=True,norm=True,norm_min_val=0):
  batch_size = y_batch.shape[0]
  N = y_batch.shape[3]
  mi_y_batch = torch.zeros(y_batch.shape)
  for i in range(batch_size):
    y = y_batch[i,:,:,:].reshape(N,N)
    y = y.reshape(-1)
    n = len(y)
    n_1 = n - 1
    z = y - y.mean()
    sy = y.std()
    z /= sy
    den = (z * z).sum()
    zl = torch.tensor(w_sparse * z)
    mi = n_1 * z * zl / den
    if na_to_zero==True:
      mi[torch.isnan(mi)] = 0
    if norm==True:
      mi = normal(mi,min_val=norm_min_val)
    mi_y_batch[i,0,:,:] = mi.reshape(N,N)
  return mi_y_batch    

#Downsampling by average pooling (needed for computing the multi-res Moran's I)
downsample = nn.AvgPool2d(kernel_size=2)

#Compute batch RMSE
def calc_rmse_batch(x,y):
  mse = nn.MSELoss()
  return torch.sqrt(mse(x,y))

Load sparse spatial weight matrices

In [ ]:
%%capture

urlretrieve('https://github.com/konstantinklemmer/sxl/raw/master/data/w/w_sparse_64.npz','w_sparse_64.npz')
urlretrieve('https://github.com/konstantinklemmer/sxl/raw/master/data/w/w_sparse_32.npz','w_sparse_32.npz')
urlretrieve('https://github.com/konstantinklemmer/sxl/raw/master/data/w/w_sparse_16.npz','w_sparse_16.npz')
urlretrieve('https://github.com/konstantinklemmer/sxl/raw/master/data/w/w_sparse_8.npz','w_sparse_8.npz')
urlretrieve('https://github.com/konstantinklemmer/sxl/raw/master/data/w/w_sparse_4.npz','w_sparse_4.npz')

In [ ]:
w_sparse_64 = scipy.sparse.load_npz('w_sparse_64.npz')
w_sparse_32 = scipy.sparse.load_npz('w_sparse_32.npz')
w_sparse_16 = scipy.sparse.load_npz('w_sparse_16.npz')
w_sparse_8 = scipy.sparse.load_npz('w_sparse_8.npz')
w_sparse_4 = scipy.sparse.load_npz('w_sparse_4.npz')

## Data

Load and prepare the DEM data. Data is normalized in the range `[0,1]`. The local Moran's I of the data can be computed at this step already, to avoid further computational burden during training.

Target (output) data is size `64 x 64`, inputs are either `32 x 32` or `16 x 16`.

In [ ]:
file_path = "list_dtm.json"
with open(file_path, 'r') as j:
  ys = np.array(json.loads(j.read()))

file_path = "list_dtm_small.json"
with open(file_path, 'r') as j:
  xs_32 = np.array(json.loads(j.read()))

file_path = "list_dtm_small2.json"
with open(file_path, 'r') as j:
  xs_16 = np.array(json.loads(j.read()))

In [ ]:
%%capture

t = ys.shape[0]
N1 = 64
N2 = 32
N3 = 16
data_ys = torch.zeros(1674,4,N1,N1)
data_xs_32 = torch.zeros(1674,2,N2,N2)
data_xs_16 = torch.zeros(1674,2,N3,N3)
for i in range(t-1):
    train_x_32_t = torch.tensor(xs_32[i,:,:])
    train_x_32_t = torch.tensor(StandardScaler().fit_transform(train_x_32_t.reshape(-1,1)))
    train_x_32_t = torch.tensor(normal(train_x_32_t.reshape(-1),min_val=0))
    train_x_16_t = torch.tensor(xs_16[i,:,:])
    train_x_16_t = torch.tensor(StandardScaler().fit_transform(train_x_16_t.reshape(-1,1)))
    train_x_16_t = torch.tensor(normal(train_x_16_t.reshape(-1),min_val=0))
    train_y_t = torch.tensor(ys[i,:,:])
    train_y_t = torch.tensor(StandardScaler().fit_transform(train_y_t.reshape(-1,1)))
    train_y_t = torch.tensor(normal(train_y_t.reshape(-1),min_val=0))
    data_xs_32[i,0,:,:] = train_x_32_t.reshape(N2,N2)
    data_xs_16[i,0,:,:] = train_x_16_t.reshape(N3,N3)
    data_ys[i,0,:,:] = train_y_t.reshape(N1,N1)
data_xs_32[:,1,:,:] = batch_lw_tensor_local_moran(data_xs_32[:,0,:,:].reshape(t,1,N2,N2),w_sparse_32,norm_min_val=0).reshape(t,N2,N2)
data_xs_16[:,1,:,:] = batch_lw_tensor_local_moran(data_xs_16[:,0,:,:].reshape(t,1,N3,N3),w_sparse_16,norm_min_val=0).reshape(t,N3,N3)
data_ys[:,1,:,:] = batch_lw_tensor_local_moran(data_ys[:,0,:,:].reshape(t,1,N1,N1),w_sparse_64,norm_min_val=0).reshape(t,N1,N1)

data_ys_d1 = downsample(data_ys[:,0,:,:].reshape(t,1,N1,N1))
data_ys_d2 = downsample(data_ys_d1)
data_mi_ys_d1 = batch_lw_tensor_local_moran(data_ys_d1[:,0,:,:].reshape(t,1,N1//2,N1//2),w_sparse_32,norm_min_val=0)
data_mi_ys_d2 = batch_lw_tensor_local_moran(data_ys_d2[:,0,:,:].reshape(t,1,N1//4,N1//4),w_sparse_16,norm_min_val=0)
data_ys[:,2,:,:] = nn.functional.interpolate(data_mi_ys_d1,scale_factor=2,mode="nearest").reshape(t,N1,N1)
data_ys[:,3,:,:] = nn.functional.interpolate(data_mi_ys_d2,scale_factor=4,mode="nearest").reshape(t,N1,N1)

## Training 

Define the model architectures the different CNN models.

In [ ]:
#Define possible models

class CNN_32(nn.Module):
    """
        CNN
    """
    def __init__(self,  nc=1, ngf=32):
        super(CNN_32, self).__init__()
        #assert INPUT_DIM[0] % 2**4 == 0, 'Should be divided 16'
        self.init_dim = (INPUT_DIM[0] // 2**4, INPUT_DIM[1] // 2**4)
        self.conv_net = nn.Sequential(nn.ConvTranspose2d(nc,ngf,kernel_size=4,stride=2,padding=1),
          nn.BatchNorm2d(ngf),
          nn.ReLU(),
          nn.ConvTranspose2d(ngf,ngf*2,kernel_size=4,stride=2,padding=1),
          nn.BatchNorm2d(ngf*2),
          nn.ReLU(),
          nn.Conv2d(ngf*2,nc,kernel_size=4,stride=2,padding=1),
          nn.Tanh()
        )
    def forward(self, x, y=None):
        y_ = x.view(x.size(0), 1, 32, 32)
        y_ = self.conv_net(y_)
        return y_

class CNN_MAT_32(nn.Module):
    """
        CNN w/ Moran's Auxiliary Task
    """
    def __init__(self,  nc=1, ngf=32):
        super(CNN_MAT_32, self).__init__()
        #assert INPUT_DIM[0] % 2**4 == 0, 'Should be divided 16'
        self.init_dim = (INPUT_DIM[0] // 2**4, INPUT_DIM[1] // 2**4)
        self.conv_net = nn.Sequential(nn.ConvTranspose2d(nc,ngf,kernel_size=4,stride=2,padding=1),
          nn.BatchNorm2d(ngf),
          nn.ReLU()
        )
        self.output_t1 = nn.Sequential(nn.ConvTranspose2d(ngf,ngf*2,kernel_size=4,stride=2,padding=1),
          nn.BatchNorm2d(ngf*2),
          nn.ReLU(),
          nn.Conv2d(ngf*2,nc,kernel_size=4,stride=2,padding=1),
          nn.Tanh()
        )
        self.output_t2 = nn.Sequential(nn.ConvTranspose2d(ngf,ngf*2,kernel_size=4,stride=2,padding=1),
          nn.BatchNorm2d(ngf*2),
          nn.ReLU(),
          nn.Conv2d(ngf*2,nc,kernel_size=4,stride=2,padding=1),
          nn.Tanh()
        )
    def forward(self, x, y=None):
        mi_x = batch_lw_tensor_local_moran(x.detach().cpu(),w_sparse_32)
        mi_x = mi_x.to(DEVICE)
        y_ = x.view(x.size(0), 1, 32, 32)
        mi_y_ = mi_x.view(mi_x.size(0), 1, 32, 32)
        y_ = self.conv_net(y_)
        mi_y_ = self.conv_net(mi_y_)
        y_ = self.output_t1(y_)
        mi_y_ = self.output_t2(mi_y_)
        return y_, mi_y_

class CNN_16(nn.Module):
    """
        CNN
    """
    def __init__(self,  nc=1, ngf=16):
        super(CNN_16, self).__init__()
        #assert INPUT_DIM[0] % 2**4 == 0, 'Should be divided 16'
        self.init_dim = (INPUT_DIM[0] // 2**4, INPUT_DIM[1] // 2**4)
        self.conv_net = nn.Sequential(nn.ConvTranspose2d(nc,ngf,kernel_size=4,stride=2,padding=1),
          nn.BatchNorm2d(ngf),
          nn.ReLU(),
          nn.ConvTranspose2d(ngf,nc,kernel_size=4,stride=2,padding=1),
          nn.Tanh()
        )
    def forward(self, x, y=None):
        y_ = x.view(x.size(0), 1, 16, 16)
        y_ = self.conv_net(y_)
        return y_

class CNN_MAT_16(nn.Module):
    """
        CNN
    """
    def __init__(self,  nc=1, ngf=16):
        super(CNN_MAT_16, self).__init__()
        #assert INPUT_DIM[0] % 2**4 == 0, 'Should be divided 16'
        self.init_dim = (INPUT_DIM[0] // 2**4, INPUT_DIM[1] // 2**4)
        self.conv_net = nn.Sequential(nn.ConvTranspose2d(nc,ngf,kernel_size=4,stride=2,padding=1),
          nn.BatchNorm2d(ngf),
          nn.ReLU()
        )
        self.output_t1 = nn.Sequential(nn.ConvTranspose2d(ngf,nc,kernel_size=4,stride=2,padding=1),
          nn.Tanh()
        )
        self.output_t2 = nn.Sequential(nn.ConvTranspose2d(ngf,nc,kernel_size=4,stride=2,padding=1),
          nn.Tanh()
        )

    def forward(self, x, y=None):
        mi_x = batch_lw_tensor_local_moran(x.detach().cpu(),w_sparse_16)
        mi_x = mi_x.to(DEVICE)
        y_ = x.view(x.size(0), 1, 16, 16)
        mi_y_ = mi_x.view(mi_x.size(0), 1, 16, 16)
        y_ = self.conv_net(y_)
        y_ = self.output_t1(y_)
        mi_y_ = self.conv_net(mi_y_)
        mi_y_ = self.output_t2(mi_y_)
        return y_, mi_y_

Define training configuration:

- `input_size`: Interpolation input size; either 16 or 32
- `model`: chose model; either Vanilla "CNN" or "CNN_MAT"
- `lambda_`: Auxiliary task weight $\lambda$
- `num_epochs`: number of training epochs 
- `batch_size`: training batch size
- `loss_fun`: loss function to be used. Default is MSE loss, but L1 or SmoothL1 can be used too.

In [ ]:
### DEFINE EXPERIMENT SETTINGS ###
input_size = "32" #Options are ["16","32"]
model = "CNN" # Options are ["CNN","CNN_MAT"]
lambda_ = 0.1 
num_epochs = 50
batch_size = 32
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # Train on GPU or CPU
loss_fun = nn.MSELoss() #Other options include e.g. nn.L1Loss() 

Define training loop and train the model.

In [ ]:
#Initiate data and prepare train / val / test split
N = int(input_size)
train_ys = data_ys[:1000,:,:,:] 
val_ys = data_ys[1000:1300,:,:,:] 
test_ys = data_ys[1300:,:,:,:] 
if input_size=="16":
  train_xs = data_xs_16[:1000,:,:,:] 
  val_xs = data_xs_16[1000:1300,:,:,:] 
  test_xs = data_xs_16[1300:,:,:,:] 
if input_size=="32":
  train_xs = data_xs_32[:1000,:,:,:] 
  val_xs = data_xs_32[1000:1300,:,:,:] 
  test_xs = data_xs_32[1300:,:,:,:] 
#Prepare data loaders
train_data = TensorDataset(train_ys, train_xs)
val_data = TensorDataset(val_ys, val_xs)
test_data = TensorDataset(test_ys, test_xs)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True, drop_last=True)
#Load models
if input_size=="16":
  if model=="CNN":
    Net = CNN_16().to(DEVICE)
  elif model=="CNN_MAT":
    Net = CNN_MAT_16().to(DEVICE)
if input_size=="32":
  if model=="CNN":
    Net = CNN_32().to(DEVICE)
  elif model=="CNN_MAT":
    Net = CNN_MAT_32().to(DEVICE)
#Utilities
rmse_val = []
check_step =[]
criterion = loss_fun
step = 0
#Initiate optimizer
Net_opt = torch.optim.Adam(Net.parameters(), lr=0.001, betas=(0.5, 0.999))
###TRAINING
print("EXPERIMENT: " + str(input_size) + " -> 64 - MODEL: " + model + " LAMBDA: " + str(lambda_))
if model=="CNN":
  for e in range(num_epochs):
    # Within each iteration, we will go over each minibatch of data
    for minibatch_i, (y_batch,x_batch) in enumerate(train_loader):
      # Get data
      x = x_batch.to(DEVICE)
      y = y_batch.to(DEVICE)
      mi_x = x[:,1,:,:].reshape(batch_size,1,N,N)
      x = x[:,0,:,:].reshape(batch_size,1,N,N)
      mi_y = y[:,1,:,:].reshape(batch_size,1,64,64)
      y = y[:,0,:,:].reshape(batch_size,1,64,64)
      # Training Net
      x_outputs = Net(x)
      Net_loss = criterion(x_outputs, y)
      Net.zero_grad()
      Net_loss.backward()
      Net_opt.step()
      # Model selection every 100 training steps
      if step % 100 == 0:
        check_step.append(step)
        torch.save(Net, "model_iter %d.pkl.gz" % step)
        rmse_batch = []
        with torch.no_grad():
            for (y_test, x_test) in val_loader:
                y_test = y_test.to(DEVICE)
                y_test = y_test[:,0,:,:].reshape(batch_size,1,64,64)
                outputs = Net(x_test[:,0,:,:].to(DEVICE).reshape(batch_size,1,N,N))
                rmse_ = calc_rmse_batch(outputs,y_test)
                rmse_batch.append(rmse_)
        rmse_val.append(torch.mean(torch.tensor(rmse_batch)))
        #Print progress
        print('Epoch [%d/%d] - Loss: %f' % (e, num_epochs, Net_loss.item()))
      #Increment steps
      step = step + 1
    #Save best and full model:
    tmp, idx = torch.min(torch.tensor(rmse_val),0)
    idx = check_step[idx]
    best_net = torch.load("model_iter " + str(idx) + ".pkl.gz")

# CNN + MAT
if model=="CNN_MAT":
  for e in range(num_epochs):
    # Within each iteration, we will go over each minibatch of data
    for minibatch_i, (y_batch,x_batch) in enumerate(train_loader):
      # Get data
      x = x_batch.to(DEVICE)
      y = y_batch.to(DEVICE)
      x = x[:,0,:,:].reshape(batch_size,1,N,N)
      mi_y = y[:,1,:,:].reshape(batch_size,1,64,64)
      y = y[:,0,:,:].reshape(batch_size,1,64,64)
      # Training Net
      x_outputs, mi_x_outputs = Net(x)
      Net_x_loss = criterion(x_outputs, y)
      Net_mi_x_loss = criterion(mi_x_outputs, mi_y)
      Net_loss = Net_x_loss + lambda_ * (Net_mi_x_loss)
      Net.zero_grad()
      Net_loss.backward()
      Net_opt.step()
      #Save model
      if step % 100 == 0:
        check_step.append(step)
        torch.save(Net, "model_iter %d.pkl.gz" % step)
        rmse_batch = []
        with torch.no_grad():
            for (y_test, x_test) in val_loader:
                y_test = y_test.to(DEVICE)
                y_test = y_test[:,0,:,:].reshape(batch_size,1,64,64)
                outputs, _ = Net(x_test[:,0,:,:].to(DEVICE).reshape(batch_size,1,N,N))
                rmse_ = calc_rmse_batch(outputs,y_test)
                rmse_batch.append(rmse_)
        rmse_val.append(torch.mean(torch.tensor(rmse_batch)))
        #Print progress
        print('Epoch [%d/%d] - Loss: %f' % (e, num_epochs, Net_loss.item()))
      #Increment steps
      step = step + 1
  #Save best and full model:
  tmp, idx = torch.min(torch.tensor(rmse_val),0)
  idx = check_step[idx]
  best_net = torch.load("model_iter " + str(idx) + ".pkl.gz")

Compute test scores

In [ ]:
rmse_scores = []

with torch.no_grad():
  for (y_test, x_test) in test_loader:
    # Get data
    x = x_test.to(DEVICE)
    y = y_test.to(DEVICE)
    mi_x = x[:,1,:,:].reshape(batch_size,1,N,N)
    x = x[:,0,:,:].reshape(batch_size,1,N,N)
    mi_y = y[:,1,:,:].reshape(batch_size,1,64,64)
    y = y[:,0,:,:].reshape(batch_size,1,64,64)
    # Training Net
    if model=="CNN":
      x_outputs = best_net(x)
    if model=="CNN_MAT":
      x_outputs, _ = best_net(x)
    rmse = calc_rmse_batch(x_outputs,y)
    rmse_scores.append(rmse)

In [ ]:
rmse_mean = torch.mean(torch.tensor(rmse_scores))

Print the final RMSE score

In [ ]:
rmse_mean